In [124]:

import yfinance as yf
import pandas as pd
import numpy as np



In [125]:
import yfinance as yf
import pandas as pd

tickers = ['^FTSE', '^GDAXI', 'FTSEMIB.MI', '^AEX', '^FCHI', '^IBEX']
index_to_country = {'^FTSE': 'UK', '^GDAXI': 'Germany', 'FTSEMIB.MI': 
                    'Italy', '^AEX': 'Netherlands', '^FCHI': 'France', '^IBEX': 'Spain'}

start_date = '2020-01-01'
end_date = '2022-12-31'


dfs = []
for index in tickers:
    data = yf.download(index, start=start_date, end=end_date)
    data['Index'] = index
    dfs.append(data)
index_data = pd.concat(dfs)
index_data['Country'] = index_data['Index'].map(index_to_country)
index_data.head(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Index,Country
Date,,,,,,,,
2020-01-02,7542.399902,7624.799805,7542.399902,7604.299805,7604.299805,482206700,^FTSE,UK
2020-01-03,7604.299805,7626.399902,7551.000000,7622.399902,7622.399902,475628900,^FTSE,UK
2020-01-06,7622.399902,7622.399902,7528.100098,7575.299805,7575.299805,516783400,^FTSE,UK
2020-01-07,7575.299805,7604.600098,7562.100098,7573.899902,7573.899902,511576100,^FTSE,UK
2020-01-08,7573.899902,7579.500000,7525.000000,7574.899902,7574.899902,606225000,^FTSE,UK
2020-01-09,7574.899902,7627.799805,7574.899902,7598.100098,7598.100098,653838100,^FTSE,UK
2020-01-10,7598.100098,7632.000000,7585.100098,7587.899902,7587.899902,651315600,^FTSE,UK
2020-01-13,7587.899902,7635.799805,7582.500000,7617.600098,7617.600098,659019400,^FTSE,UK
2020-01-14,7617.600098,7650.700195,7588.799805,7622.399902,7622.399902,697648200,^FTSE,UK


In [153]:
import pandas as pd


election_data_file = 'C:\\Users\\rafme\\Downloads\\Election_data.xlsx'
election_data = df_election = pd.read_excel(election_data_file)
election_data.head()




,Date,Winner,Party,Spectrum,Share,Country
0,2021-09-26,Scholz,SPD,centre-left,25.7,Germany
1,2017-09-24,Merkel,CDU,centre,32.9,Germany
2,2013-09-22,Merkel,CDU,centre,41.5,Germany
3,2009-09-27,Merkel,CDU,centre-right,33.8,Germany
4,2005-09-18,Merkel,CDU,centre,35.2,Germany


In [144]:
# Function to calculate optimal investment plan
def calculate_optimal_investment_plan(df, election_df, window_before=182, window_after=182):
    ''' 
    Calculate the optimal investment plan (days before and after the election) for each country.

    Parameters:
    - df: DataFrame containing historical stock data with 'Close' prices and a 'Country' column.
    - election_df: DataFrame containing election information with 'Date' column.
    - window_before: Maximum number of days before the election to consider.
    - window_after: Maximum number of days after the election to consider.

    Returns:
    - results: A list of dictionaries containing information about the optimal investment time for each country.
      Each dictionary has keys: 'Country', 'Election_Date', 'Buy_Days_Before', 'Sell_Days_After', 'Profit_Percentage'.
    '''
    # Get unique countries from the stock DataFrame
    unique_countries = df['Country'].unique()

    # Initialize a list to store results for each country
    results = []

    for country in unique_countries:
        # Filter stock data for the current country
        country_data = df[df['Country'] == country]

        for _, election_row in election_df[election_df['Country'] == country].iterrows():
            # Election date for the current country
            election_date = election_row['Date']

            # Skip rows with 'NaT' (Not a Time) values
            if pd.isna(election_date):
                continue

            # Calculate start and end dates for the analysis window
            start_date = election_date - pd.DateOffset(days=window_before)
            end_date = election_date + pd.DateOffset(days=window_after)

            # Filter stock data for the analysis window
            stock_data = country_data[(country_data['Date'] >= start_date) & (country_data['Date'] <= end_date)]

            max_profit = 0
            buy_days_before = 0
            sell_days_after = 0
            
            for i in range(len(stock_data) - 1):
                for j in range(i + 1, len(stock_data)):
                    profit = (stock_data['Close'].iloc[j] - stock_data['Close'].iloc[i]) / stock_data['Close'].iloc[i]
                    if profit > max_profit:
                        max_profit = profit
                        buy_days_before = (stock_data.index[i] - election_date).days
                        sell_days_after = (stock_data.index[j] - election_date).days

            # Calculate profit percentage
            buy_price = stock_data.iloc[0]['Close']  # First day of the election window
            sell_price = stock_data.iloc[-1]['Close']  # Last day of the election window
            profit_percentage = ((sell_price - buy_price) / buy_price) * 100

            result_dict = {
                'Country': country,
                'Election_Date': election_date,
                'Buy_Days_Before': buy_days_before,
                'Sell_Days_After': sell_days_after,
                'Profit_Percentage': profit_percentage
            }

            results.append(result_dict)

    return results


In [151]:
election_data_file = 'C:\\Users\\rafme\\Downloads\\Election_data.xlsx'
election_data = df_election = pd.read_excel(election_data_file)
election_data['Date'] = pd.to_datetime(election_data['Date'], errors='coerce')

In [ ]:
def calculate_max_profit(df):
    ''' 
    Calculate the maximum profit that can be made by buying and selling a stock.

    Parameters:
    - df: DataFrame containing historical stock data with 'Close' prices.

    Returns:
    - max_profit: Maximum profit that can be made.
    - buy_date: Date to buy the stock for maximum profit.
    - sell_date: Date to sell the stock for maximum profit'''
    max_profit = 0
    buy_date = None
    sell_date = None
    window_size = 12
    
    for i in range(len(df) - window_size):
        for j in range(i + window_size, len(df)):
            profit = (df['Close'].iloc[j] - df['Close'].iloc[i])/df['Close'].iloc[i]
            if profit > max_profit:
                max_profit = profit
                buy_date = df.index[i]
                sell_date = df.index[j]

    return max_profit, buy_date, sell_date

for index in tickers:
    subset = index_data[index_data['Index'] == index]
    max_profit, buy_date, sell_date = calculate_max_profit(subset)
    print(f"Index: {index}, Max Profit: {max_profit*100:.2f}%, Buy Date: {buy_date}, Sell Date: {sell_date} ")
